[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/openlayer-ai/examples-gallery/blob/main/monitoring/quickstart/traditional-ml/monitoring-quickstart.ipynb)


# <a id="top">Monitoring quickstart</a>

This notebook illustrates a typical monitoring flow using Openlayer. For more details, refer to the [How to set up monitoring guide](https://docs.openlayer.com/documentation/how-to-guides/set-up-monitoring) from the documentation.


## <a id="toc">Table of contents</a>

1. [**Creating a project and an inference pipeline**](#inference-pipeline) 

2. [**Publishing batches of production data**](#publish-batches)

3. [(Optional) **Uploading a reference dataset**](#reference-dataset)

4. [(Optional) **Publishing ground truths**](#ground-truths)

Before we start, let's download the sample data and import pandas.

In [1]:
%%bash

if [ ! -e "churn_train.csv" ]; then
    curl "https://openlayer-static-assets.s3.us-west-2.amazonaws.com/examples-datasets/monitoring/churn_train.csv" --output "churn_train.csv"
fi

if [ ! -e "prod_data_no_ground_truths.csv" ]; then
    curl "https://openlayer-static-assets.s3.us-west-2.amazonaws.com/examples-datasets/monitoring/prod_data_no_ground_truths.csv" --output "prod_data_no_ground_truths.csv"
fi

if [ ! -e "prod_ground_truths.csv" ]; then
    curl "https://openlayer-static-assets.s3.us-west-2.amazonaws.com/examples-datasets/monitoring/prod_ground_truths.csv" --output "prod_ground_truths.csv"
fi

In [1]:
import pandas as pd

## <a id="inference-pipeline"> 1. Creating a project and an inference pipeline </a>

[Back to top](#top)

In [ ]:
!pip install openlayer

In [2]:
import openlayer

openlayer.api.OPENLAYER_ENDPOINT = "http://localhost:8080/v1"
openlayer.api.STORAGE = openlayer.api.StorageType.ONPREM

client = openlayer.OpenlayerClient("-feQN48QZ9haCfq80m3zcpUTndHhjf-A")

In [3]:
from openlayer.tasks import TaskType

project = client.create_project(
    name="Churn Prediction 2",
    task_type=TaskType.TabularClassification,
)

Created your project. Navigate to http://localhost:8000/w-33c53ff/107dec23-c941-44e8-b63d-40dc76073a07 to see it.


Now that you are authenticated and have a project on the platform, it's time to create an inference pipeline. Creating an inference pipeline is what enables the monitoring capabilities in a project.

In [4]:
inference_pipeline = project.create_inference_pipeline()

Created your inference pipeline. Navigate to http://localhost:8000/w-33c53ff/107dec23-c941-44e8-b63d-40dc76073a07?mode=monitoring&inferencePipelineId=dc75c134-09f2-47c3-b645-9d61fb10c7b6 to see it.


## <a id="publish-batches"> 2. Publishing production data </a>

[Back to top](#top)

In production, as the model makes predictions, the data can be published to Openlayer. This is done with the `publish_batch_data` method. 

The data published to Openlayer can have a column with **inference ids** and another with **timestamps** (UNIX sec format). These are both optional and, if not provided, will receive default values. The inference id is particularly important if you wish to publish ground truths at a later time. 

In [5]:
production_data = pd.read_csv("prod_data_no_ground_truths.csv")

In [6]:
batch_1 = production_data.loc[:342]
batch_2 = production_data.loc[343:684]
batch_3 = production_data.loc[686:]

In [7]:
batch_1.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,AggregateRate,Year,predictions,inference_id,timestamp
0,796,Spain,Male,56.0,6,94231.13,1,0,0,121164.60,2.52,2023,1,2edc5eb5-ff0d-448a-acf6-7543b9252fc6,1694361665
1,733,France,Male,39.0,1,0.00,2,1,1,141841.31,0.42,2023,0,786c8682-9261-4345-8004-98736624a850,1694361665
2,556,France,Male,34.0,8,163757.06,1,1,1,104000.06,3.36,2023,0,60b2a8b5-d936-4329-a5db-25c9a36f6a7a,1694361665
3,489,Spain,Male,58.0,4,0.00,2,1,1,191419.32,1.68,2023,0,2860bb85-1469-401a-9009-10ce0070d6e7,1694361665
4,639,France,Male,22.0,4,0.00,2,1,0,28188.96,1.68,2023,0,90d128f1-2797-45d9-8106-619e4bd09889,1694361665


### <a id="publish-batches"> Publish to Openlayer </a>

Here, we're simulating three calls to `publish_batch_data`. In practice, this is a code snippet that lives in your inference pipeline and that gets called after the model predictions.

In [8]:
batch_config = {
    "categoricalFeatureNames": ["Gender", "Geography"],
    "classNames": ["Retained", "Exited"],
    "featureNames": [
        "CreditScore", 
        "Geography",
        "Gender",
        "Age", 
        "Tenure",
        "Balance",
        "NumOfProducts",
        "HasCrCard",
        "IsActiveMember",
        "EstimatedSalary",
        "AggregateRate",
        "Year"
    ],
    "timestampColumnName": "timestamp",
    "inferenceIdColumnName": "inference_id",
    "predictionsColumnName": "predictions"
}


In [9]:
inference_pipeline.publish_batch_data(
    batch_df=batch_1,
    batch_config=batch_config
)

Data published!


In [10]:
inference_pipeline.publish_batch_data(
    batch_df=batch_2,
    batch_config=batch_config
)

Data published!


In [14]:
stream_data = batch_3.iloc[:2, :].to_dict(orient="records")

In [15]:
stream_data

[{'CreditScore': 545,
  'Geography': 'France',
  'Gender': 'Female',
  'Age': 26.0,
  'Tenure': 1,
  'Balance': 0.0,
  'NumOfProducts': 2,
  'HasCrCard': 1,
  'IsActiveMember': 1,
  'EstimatedSalary': 199638.56,
  'AggregateRate': 0.42,
  'Year': 2022,
  'predictions': 0,
  'inference_id': '8d79c28e-20db-4fb9-bee1-9efe01b414dd',
  'timestamp': 1694620865},
 {'CreditScore': 771,
  'Geography': 'Germany',
  'Gender': 'Female',
  'Age': 36.0,
  'Tenure': 5,
  'Balance': 77846.9,
  'NumOfProducts': 1,
  'HasCrCard': 0,
  'IsActiveMember': 0,
  'EstimatedSalary': 99805.99,
  'AggregateRate': 2.1,
  'Year': 2022,
  'predictions': 1,
  'inference_id': '0fc0f4c1-4e73-4488-afbe-79e93a5a0509',
  'timestamp': 1694620865}]

In [16]:
inference_pipeline.stream_data(
    stream_data=stream_data,
    stream_config=batch_config
)

OpenlayerInvalidRequest: <Response [400]> {'categoricalFeatureNames': ['Gender', 'Geography'], 'label': 'production', 'metadata': {}, 'predictionScoresColumnName': None, 'language': 'en', 'columnNames': ['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'AggregateRate', 'Year', 'predictions', 'inference_id', 'timestamp'], 'predictionsColumnName': 'predictions', 'inferenceIdColumnName': 'inference_id', 'sep': ',', 'classNames': ['Retained', 'Exited'], 'featureNames': ['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'AggregateRate', 'Year'], 'timestampColumnName': 'timestamp', 'labelColumnName': None, 'latencyColumnName': None} is not valid under any of the given schemas - 'config'

**That's it!** You're now able to set up tests and alerts for your production data. The next sections are optional and enable some features on the platform.

## <a id="reference-dataset"> 3. Uploading a reference dataset </a>

[Back to top](#top)

A reference dataset is optional, but it enables drift monitoring. Ideally, the reference dataset is a representative sample of the training set used to train the deployed model. In this section, we first load the dataset and then we upload it to Openlayer using the `upload_reference_dataframe` method.

In [ ]:
training_set = pd.read_csv("./churn_train.csv")

### <a id="upload-reference"> Uploading the dataset to Openlayer </a>

In [ ]:
dataset_config = {
    "categoricalFeatureNames": ["Gender", "Geography"],
    "classNames": ["Retained", "Exited"],
        "featureNames": [
        "CreditScore", 
        "Geography",
        "Gender",
        "Age", 
        "Tenure",
        "Balance",
        "NumOfProducts",
        "HasCrCard",
        "IsActiveMember",
        "EstimatedSalary",
        "AggregateRate",
        "Year"
    ],
    "labelColumnName": "Exited",
    "label": "reference"
}

In [ ]:
inference_pipeline.upload_reference_dataframe(
    dataset_df=training_set,
    dataset_config=dataset_config
)

## <a id="ground-truths"> 4. Publishing ground truths for past batches </a>

[Back to top](#top)

The ground truths are needed to create Performance tests. The `publish_ground_truths` method can be used to update the ground truths for batches of data already published to the Openlayer platform. The inference id is what gets used to merge the ground truths with the corresponding rows.

In [ ]:
ground_truths = pd.read_csv("prod_ground_truths.csv")

### <a id="publish-truth">Publish ground truths </a>

In [ ]:
inference_pipeline.publish_ground_truths(
    df=ground_truths,
    ground_truth_column_name="Exited",
    inference_id_column_name="inference_id",
)